In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np
import albumentations as A
from glob import glob
import os

In [ ]:
def get_augmentations(category):
    if category == "healthy":
        return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.Affine(scale=(0.9, 1.1), translate_percent=(0.02, 0.02), rotate=(-20, 20), p=0.7),
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5)
        ])
    elif category == "bleached":
        return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.Affine(scale=(0.9, 1.1), translate_percent=(0.02, 0.02), rotate=(-20, 20), p=0.7),
            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.05, p=0.5),
            A.GaussianBlur(blur_limit=(3, 5), p=0.2)
        ])
    else:  # Diseased
        return A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.2),
            A.Affine(scale=(0.9, 1.1), translate_percent=(0.02, 0.02), rotate=(-20, 20), p=0.7),
            A.RandomBrightnessContrast(brightness_limit=-0.2, contrast_limit=0.3, p=0.5),
            A.MotionBlur(blur_limit=(3, 5), p=0.2),
            A.Perspective(scale=(0.02, 0.05), p=0.3),
            A.ElasticTransform(alpha=1.0, sigma=50.0, p=0.2)
        ])


In [ ]:
def augment_image(image_path, save_dir, num_augments, augmenter):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Save original image
    cv2.imwrite(os.path.join(save_dir, 'original_' + os.path.basename(image_path)), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

    # Apply augmentations
    for i in range(1, num_augments + 1):
        augmented_img = augmenter(image=image)['image']
        cv2.imwrite(os.path.join(save_dir, f'augmented_{i}_' + os.path.basename(image_path)), cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR))

In [ ]:
# Define dataset categories
categories = {
    "diseased": {"input_folder": "/content/drive/MyDrive/disease", "output_folder": "/content/drive/MyDrive/coral_data/diseased", "num_augments": 7},
    "healthy": {"input_folder": "/content/drive/MyDrive/healthy_corals", "output_folder": "/content/drive/MyDrive/coral_data/healthy", "num_augments": 3},
    "bleached": {"input_folder": "/content/drive/MyDrive/bleached_corals", "output_folder": "/content/drive/MyDrive/coral_data/bleached", "num_augments": 2},
}

In [ ]:
# Process images for each category
for category, settings in categories.items():
    os.makedirs(settings["output_folder"], exist_ok=True)
    image_paths = glob(os.path.join(settings["input_folder"], "*.jpg"))
    augmenter = get_augmentations(category)
    for img_path in image_paths:
        augment_image(img_path, settings["output_folder"], settings["num_augments"], augmenter)